### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt

from skimage.transform import resize
from keras.models import model_from_json
import tensorflow as tf

from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50 
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


In [4]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    '''
    Reads in a .dcm file, checks the important fields for our device and returns a numpy array of imaging data.
    Input:
    filename - .dcm filename
    img - numpy array of imaging data
    bodypartexamined - string with name of body part examined
    '''
    
    # get imaging data array
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    body_part = ds.BodyPartExamined
    
    # check important fields:
    print("Body part examined: {}".format(ds.BodyPartExamined))
    print("Image modality: {}".format(ds.Modality))
    print("Disease finding: {}".format(ds.StudyDescription))
    print("Patient ID: {}".format(ds.PatientID))
    print("Patient Sex: {}".format(ds.PatientSex))
    print("Patient Age: {}".format(ds.PatientAge))
    print("Image rows: {}".format(ds.Rows))
    print("Image columns: {}".format(ds.Columns))

    return img, body_part
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    '''
    Funtion to resize image and standardize image before predicting.
    Input:
    img - image to predict against.
    img_mean - mean of image from training data
    img_std - standard deviation of image from training data
    img_size - size of image
    
    Output: 
    proc_img - processed imamge
    '''

   
    # resize image
    read_img =  resize(img, img_size)
    
    #print(read_img.shape)
    
    # standardize image
    proc_img = (read_img - img_mean)/img_std
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    '''
    Funtion to load and compile the trained model with weights.
    Input: 
    model_path - path to saved model
    weight_path - path to saved weights file
    
    Output:
    model - compiled model
    '''
    
    # load json and create model
    json_file = open(model_path, 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    
    # load weights into new model
    model.load_weights(weight_path)
    
    # Set our optimizer, loss function, and learning rate
    optimizer = Adam(lr=1e-4)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    #model.summary()
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    '''
    Funtion to predict presence of pneumonia in an image using threshold parameter.
    Input:
    model - compiled model
    thresh - threshold parameter
    
    Output:
    prediction - prediction of presence of pneumonia
    '''
    
    pred_Y = model.predict(img, verbose = True)
    
    if (pred_Y>thresh):
        prediction = "Pneumonia"
    else:
        prediction = "No Pneumonia"
    
    return prediction 

In [7]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'
weight_path = 'xray_class_my_model.best.hdf5'

IMG_SIZE = [1,224,224,3] # This might be different if you did not use vgg16
img_mean = 124.93775177001953 # loads the mean image value they used during training preprocessing
img_std = 63.31730651855469 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.6 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img, body_part_ex = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc  = preprocess_image(img, img_mean, img_std, IMG_SIZE)
    if (body_part_ex == 'CHEST'):
        pred = predict_image(my_model,img_proc,thresh)
        print("Prediction using model: {}".format(pred))
        print("------------------------")
    else:
        print("The body part examined is not Chest.")
        print("------------------------")

Load file test1.dcm ...
Body part examined: CHEST
Image modality: DX
Disease finding: No Finding
Patient ID: 2
Patient Sex: M
Patient Age: 81
Image rows: 1024
Image columns: 1024
1/1 [==============================] - 1s 647ms/step
Prediction using model: No Pneumonia
------------------------
Load file test2.dcm ...
Body part examined: CHEST
Image modality: DX
Disease finding: Cardiomegaly
Patient ID: 1
Patient Sex: M
Patient Age: 58
Image rows: 1024
Image columns: 1024
1/1 [==============================] - 1s 587ms/step
Prediction using model: No Pneumonia
------------------------
Load file test3.dcm ...
Body part examined: CHEST
Image modality: DX
Disease finding: Effusion
Patient ID: 61
Patient Sex: M
Patient Age: 77
Image rows: 1024
Image columns: 1024
1/1 [==============================] - 1s 585ms/step
Prediction using model: No Pneumonia
------------------------
Load file test4.dcm ...
Body part examined: RIBCAGE
Image modality: DX
Disease finding: No Finding
Patient ID: 2
Pati